In [84]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras import backend as K
from NLHoldem import Game, Deck
from HandComperator import compare_hands
from keras.utils import to_categorical

In [8]:
game = Game()
deck = Deck()

In [63]:
n_hands = 1000000
player_1_hands = []
player_2_hands = []
boards = []
strengths = []
for i in range(n_hands):
    deck.shuffle()
    player_1 = deck[:2]
    player_2 = deck[2:4]
    player_1_hands.append([[player_1[0].value,player_1[0].suit],[player_1[1].value,player_1[1].suit]])
    player_2_hands.append([[player_2[0].value,player_2[0].suit],[player_2[1].value,player_2[1].suit]])
    board = deck[4:9]
    player_1 = player_1 +board
    player_2 = player_2 +board
    strengths.append(compare_hands([player_1,player_2]))
    boards.append([[card.value,card.suit] for card in board])


In [64]:
player_1_array = np.array(player_1_hands).reshape((n_hands,4))
player_2_array = np.array(player_2_hands).reshape((n_hands,4))
boards_array = np.array(boards).reshape(n_hands,10)
strengths_array = np.array(strengths).reshape(n_hands,2)
data = pd.DataFrame(columns=["p1_c_1_val","p1_c_1_suit","p1_c_2_val","p1_c_2_suit","p2_c_1_val","p2_c_1_suit","p2_c_2_val","p2_c_2_suit",
                             "f_1_val","f_1_suit","f_2_val","f_2_suit","f_3_val","f_3_suit","t_val","t_suit","r_val","r_suit","strength_1", "strength_2"])

In [65]:
data[["p1_c_1_val","p1_c_1_suit","p1_c_2_val","p1_c_2_suit"]] = player_1_array
data[["p2_c_1_val","p2_c_1_suit","p2_c_2_val","p2_c_2_suit"]] = player_2_array
data[["f_1_val","f_1_suit","f_2_val","f_2_suit","f_3_val","f_3_suit","t_val","t_suit","r_val","r_suit"]]=boards_array
data[["strength_1","strength_2"]]= strengths_array

In [66]:
def find_result(a,b):
    if a == b:
        return 0
    elif a > b:
        return 1
    else:
        return 2
data["result"] = data.apply(lambda row: find_result(row["strength_1"], row["strength_2"]), axis=1)
data

,p1_c_1_val,p1_c_1_suit,p1_c_2_val,p1_c_2_suit,p2_c_1_val,p2_c_1_suit,p2_c_2_val,p2_c_2_suit,f_1_val,f_1_suit,...,f_2_suit,f_3_val,f_3_suit,t_val,t_suit,r_val,r_suit,strength_1,strength_2,result
0,13,1,5,2,7,0,4,1,8,2,...,1,7,2,12,2,12,1,1.121308,2.120708,2
1,2,2,9,0,4,3,12,2,9,1,...,3,14,2,9,2,10,1,3.091413,1.091413,1
2,4,3,11,3,3,3,11,2,11,1,...,1,14,0,13,1,8,3,2.141113,2.141113,0
3,2,2,13,1,6,1,5,0,11,1,...,3,9,3,14,1,7,3,1.141311,1.141109,1
4,6,1,5,0,7,3,3,2,11,1,...,0,14,1,8,3,10,2,0.141110,0.141110,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,7,3,8,0,9,0,9,1,3,1,...,2,14,3,5,1,5,0,1.051408,2.090514,2
999996,12,2,9,3,6,1,14,2,13,2,...,2,4,1,6,3,14,0,0.141312,2.140613,2
999997,7,1,14,1,12,0,13,3,12,3,...,1,13,0,12,1,10,0,1.121413,6.121300,2
999998,6,1,3,0,14,0,8,1,11,0,...,0,5,1,10,2,14,3,0.141110,1.141110,2


In [88]:
input_dims = 18

fc1_dims = 128
fc2_dims = 64
fc3_dims = 32
fc4_dims = 16
fc5_dims = 8
fc6_dims = 4

lr = 0.0001
model= keras.Sequential()
model.add(keras.layers.Dense(fc1_dims, input_dim=input_dims, activation = 'relu'))
model.add(keras.layers.Dense(fc2_dims, activation = 'relu'))
model.add(keras.layers.Dense(fc3_dims, activation = 'relu'))
model.add(keras.layers.Dense(fc4_dims, activation = 'relu'))
model.add(keras.layers.Dense(fc5_dims, activation = 'relu'))
model.add(keras.layers.Dense(fc6_dims, activation = 'relu'))
model.add(keras.layers.Dense(3,activation='softmax')) #pwin, ploose and pplit must add up to 1.
model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy')

In [87]:
X_train = np.array(data.iloc[:970000,:18])
Y_train = np.array(data.iloc[:970000,-1])
Y_train = to_categorical(Y_train)
X_test = np.array(data.iloc[970000:,:18])
Y_test = np.array(data.iloc[970000:,-1])
Y_test = to_categorical(Y_test)

In [105]:
model.fit(X_train, Y_train, epochs = 200, batch_size=64)

Epoch 1/200
15157/15157 [==============================] - 37s 2ms/step - loss: 0.4827
Epoch 2/200
15157/15157 [==============================] - 36s 2ms/step - loss: 0.4821
Epoch 3/200
15157/15157 [==============================] - 36s 2ms/step - loss: 0.4818
Epoch 4/200
15157/15157 [==============================] - 36s 2ms/step - loss: 0.4810
Epoch 5/200
15157/15157 [==============================] - 36s 2ms/step - loss: 0.4804
Epoch 6/200
15157/15157 [==============================] - 37s 2ms/step - loss: 0.4801
Epoch 7/200
15157/15157 [==============================] - 37s 2ms/step - loss: 0.4796
Epoch 8/200
15157/15157 [==============================] - 37s 2ms/step - loss: 0.4788
Epoch 9/200
15157/15157 [==============================] - 38s 2ms/step - loss: 0.4785
Epoch 10/200
15157/15157 [==============================] - 37s 2ms/step - loss: 0.4775
Epoch 11/200
15157/15157 [==============================] - 38s 2ms/step - loss: 0.4771
Epoch 12/200
15157/15157 [===============

KeyboardInterrupt: 

In [118]:
model.predict(X_test)

938/938 [==============================] - 1s 1ms/step


array([[0.00930468, 0.37504953, 0.61564577],
       [0.00239031, 0.90693474, 0.09067492],
       [0.10098772, 0.57427496, 0.32473734],
       ...,
       [0.00408461, 0.13857135, 0.857344  ],
       [0.00119526, 0.04585903, 0.95294577],
       [0.08170473, 0.39796764, 0.5203277 ]], dtype=float32)

In [107]:
Y_test

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [108]:
model.save('comparator.h5')

In [103]:
model= load_model('comparator.h5')

In [119]:
np.equal(np.argmax(prediction,axis=1),np.array(data.iloc[970000:,-1])).sum()

24745